In [1]:
import pickle
import os
def f(dict, path):
    path = os.path.join(path, dict)
    with open(path, 'rb') as f:
        return pickle.load(f)

path = os.path.join("emoji_data")
files = os.listdir(path)
loaded_dicts = [f(file, path) for file in files]

In [2]:
dics_names = [
    "EMOJI_UNICODE",
    "UNICODE_EMOJI",
    "EMOJI_ALIAS_UNICODE",
    "UNICODE_EMOJI_ALIAS",
    "EMOTICONS_EMO",
]

In [3]:
import numpy as np
import pandas as pd

In [4]:
keys = np.array(list(loaded_dicts[3].keys()))
print(keys.shape)

(3521,)


In [5]:
def find_indeces(columns, values):
    columns = columns.flatten()
    inds = np.array([np.where(columns == searchval)[0] for searchval in values]).flatten()
    return inds

In [6]:
main = pd.read_csv("dataset_.csv")[["emoi","label"]]

dropped_empty = main[main.emoi != "[]"]
import ast
dropped_empty['emoi']=dropped_empty['emoi'].apply(ast.literal_eval)
emojis_set = dropped_empty.emoi.to_list()
encoded = np.zeros((len(emojis_set),keys.shape[0]))
for row, emojis in zip(encoded, emojis_set):
    row[find_indeces(keys,emojis)] = 1
encoded_df = pd.DataFrame(encoded)
dropped_empty.reset_index(drop=True, inplace=True)
dropped_empty.index
merged = dropped_empty.join(encoded_df)
merged.drop(columns="emoi", inplace=True)

<ipython-input-6-bc49e4fd9974>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropped_empty['emoi']=dropped_empty['emoi'].apply(ast.literal_eval)


In [7]:
merged

,label,0,1,2,3,4,5,6,7,8,...,3511,3512,3513,3514,3515,3516,3517,3518,3519,3520
0,happy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,happy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,happy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,happy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,happy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2583,disappointed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2584,disappointed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2585,disappointed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2586,disappointed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
y = merged.label

In [9]:
merged.iloc[:, 1:].sum().sum()

4020.0

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [11]:
X = merged.iloc[:, 1:]
y = merged.iloc[:, 0]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [14]:
sum(model.predict(X_test) == y_test) / len(y_test)

0.5907335907335908

In [15]:
model.predict(X_test)

array(['happy', 'disappointed', 'sad', 'happy', 'sad', 'happy', 'happy',
       'happy', 'happy', 'disappointed', 'angry', 'disappointed',
       'disappointed', 'disappointed', 'disappointed', 'sad', 'happy',
       'happy', 'disappointed', 'happy', 'angry', 'disappointed', 'sad',
       'disappointed', 'disappointed', 'disappointed', 'happy',
       'disappointed', 'angry', 'happy', 'disappointed', 'sad', 'happy',
       'disappointed', 'angry', 'happy', 'disappointed', 'disappointed',
       'angry', 'happy', 'sad', 'sad', 'angry', 'sad', 'angry', 'happy',
       'happy', 'happy', 'happy', 'happy', 'angry', 'happy', 'angry',
       'happy', 'disappointed', 'disappointed', 'happy', 'disappointed',
       'happy', 'disappointed', 'angry', 'happy', 'happy', 'angry',
       'happy', 'happy', 'sad', 'angry', 'happy', 'sad', 'happy', 'happy',
       'disappointed', 'sad', 'disappointed', 'disappointed',
       'disappointed', 'happy', 'disappointed', 'happy', 'sad', 'happy',
       'happy

In [16]:
y_test

691            happy
1926           angry
1048             sad
2106    disappointed
1216             sad
            ...     
739            happy
1661           angry
340            happy
1888           angry
1803           angry
Name: label, Length: 518, dtype: object

In [17]:
merged.iloc[-100:, 0]

2488    disappointed
2489    disappointed
2490    disappointed
2491    disappointed
2492    disappointed
            ...     
2583    disappointed
2584    disappointed
2585    disappointed
2586    disappointed
2587    disappointed
Name: label, Length: 100, dtype: object

In [18]:
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

model = keras.models.Sequential()
model.add(keras.layers.Input((None, X_train.shape[1])))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(4, activation='softmax'))

model.compile(optimizer='adam', metrics=['acc'], loss='bce')

In [19]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(pd.DataFrame(y_train))

OneHotEncoder(handle_unknown='ignore')

In [20]:
y_train_enc = encoder.transform(pd.DataFrame(y_train)).toarray()
y_test_enc = encoder.transform(pd.DataFrame(y_test)).toarray()

In [21]:
y_train_enc.shape

(2070, 4)

In [22]:
model.fit(X_train, y_train_enc, validation_data=(X_test, y_test_enc), epochs=30)

Epoch 1/30
65/65 [==============================] - 1s 10ms/step - loss: 0.5643 - acc: 0.3821 - val_loss: 0.4761 - val_acc: 0.5212
Epoch 2/30
65/65 [==============================] - 0s 7ms/step - loss: 0.4564 - acc: 0.4773 - val_loss: 0.3799 - val_acc: 0.6023
Epoch 3/30
65/65 [==============================] - 0s 7ms/step - loss: 0.4332 - acc: 0.5058 - val_loss: 0.3638 - val_acc: 0.6293
Epoch 4/30
65/65 [==============================] - 0s 6ms/step - loss: 0.4302 - acc: 0.5000 - val_loss: 0.3508 - val_acc: 0.6216
Epoch 5/30
65/65 [==============================] - 0s 6ms/step - loss: 0.4163 - acc: 0.5295 - val_loss: 0.3438 - val_acc: 0.6313
Epoch 6/30
65/65 [==============================] - 0s 6ms/step - loss: 0.4160 - acc: 0.5097 - val_loss: 0.3441 - val_acc: 0.6332
Epoch 7/30
65/65 [==============================] - 0s 6ms/step - loss: 0.4146 - acc: 0.5135 - val_loss: 0.3429 - val_acc: 0.6158
Epoch 8/30
65/65 [==============================] - 0s 6ms/step - loss: 0.4121 - acc: 0.4

In [23]:
X_train.shape

(2070, 3521)

In [24]:
y_train_enc.shape

(2070, 4)

In [25]:
model.save(filepath='emoticons.h5')